In [3]:
import os
import csv
import numpy as np
from plyfile import PlyData
from scipy.spatial.distance import directed_hausdorff

class PointCloudComparer:
    def __init__(self, sample_dir, output_dir):
        self.sample_dir = sample_dir
        self.output_dir = output_dir

    def load_pointcloud(self, dir, filename):
        plydata = PlyData.read(os.path.join(dir, filename))
        pc = np.vstack([plydata['vertex']['x'], plydata['vertex']['y'], plydata['vertex']['z']]).T
        return pc

    def get_binary_image(self, pc, axes, min_coordinates, max_coordinates):
        pc_rounded = np.round(pc[:, axes]).astype(int)
        pc_shifted = pc_rounded - min_coordinates
        shape = max_coordinates - min_coordinates + 1
        img = np.zeros(shape)
        img[pc_shifted[:,0], pc_shifted[:,1]] = 1  # y first, then x
        return img

    def compute_metrics(self, img1_bin, img2_bin):
        # Compute pixel accuracy
        pixel_accuracy = np.sum(img1_bin == img2_bin) / np.prod(img1_bin.shape)

        # Compute Intersection over Union (IoU)
        intersection = np.logical_and(img1_bin, img2_bin)
        union = np.logical_or(img1_bin, img2_bin)
        iou = np.sum(intersection) / np.sum(union)

        # Compute Dice Similarity Coefficient (DSC)
        dsc = 2 * np.sum(intersection) / (np.sum(img1_bin) + np.sum(img2_bin))

        # Compute Mean Squared Error (MSE)
        mse = np.sum((img1_bin - img2_bin) ** 2) / np.prod(img1_bin.shape)

        return pixel_accuracy, iou, dsc, mse

    def process_group(self, group_number):
        metrics = {}
        pc1 = self.load_pointcloud(self.sample_dir, f"T{group_number}_M.ply")
        pc2 = self.load_pointcloud(self.sample_dir, f"T{group_number}_T.ply")

        # Ensure that both point clouds are 3D
        pc1 = pc1[:, :3]
        pc2 = pc2[:, :3]

        views = [('front', [0, 1]), ('side', [1, 2]), ('top', [0, 2])]

        for view, axes in views:
            pc1_rounded = np.round(pc1[:, axes]).astype(int)
            pc2_rounded = np.round(pc2[:, axes]).astype(int)

            min_coordinates = np.min(np.vstack([pc1_rounded, pc2_rounded]), axis=0)
            max_coordinates = np.max(np.vstack([pc1_rounded, pc2_rounded]), axis=0)

            img1_bin = self.get_binary_image(pc1, axes, min_coordinates, max_coordinates)
            img2_bin = self.get_binary_image(pc2, axes, min_coordinates, max_coordinates)

            pixel_accuracy, iou, dsc, mse = self.compute_metrics(img1_bin, img2_bin)

            metrics[f"{view}_pixel_accuracy"] = pixel_accuracy
            metrics[f"{view}_IoU"] = iou
            metrics[f"{view}_DSC"] = dsc
            metrics[f"{view}_MSE"] = mse

        hausdorff_distance = directed_hausdorff(pc1, pc2)[0]
        metrics["Hausdorff_distance"] = hausdorff_distance

        min_coordinates = np.min(np.vstack([pc1, pc2]), axis=0)
        max_coordinates = np.max(np.vstack([pc1, pc2]), axis=0)
        diagonal_length = np.linalg.norm(max_coordinates - min_coordinates)
        normalized_hausdorff_distance = hausdorff_distance / diagonal_length
        metrics["Normalized_Hausdorff_distance"] = normalized_hausdorff_distance

        return metrics

    def process_all(self, total_samples):
        header = ["sample", "front_pixel_accuracy", "front_IoU", "front_DSC", "front_MSE",
                  "side_pixel_accuracy", "side_IoU", "side_DSC", "side_MSE",
                  "top_pixel_accuracy", "top_IoU", "top_DSC", "top_MSE",
                  "Hausdorff_distance", "Normalized_Hausdorff_distance"]
        with open(os.path.join(self.output_dir, "results.csv"), "w", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(header)

            for i in range(148, 148 + total_samples):
                print(f"Processing sample T{i}...")
                metrics = self.process_group(i)
                row = [f'T{i}'] + [metrics[metric] for metric in header[1:]]
                writer.writerow(row)


In [2]:
# Define the directories
sample_dir = "E:\A files\Master\Dissertation\Samples\MLA01\MLA01_ply"
output_dir = "E:\A files\Master\Dissertation\Samples\MLA01"

# Initialize and run the comparer
total_samples = 147
comparer = PointCloudComparer(sample_dir, output_dir)
comparer.process_all(total_samples)

Processing sample T1...
Processing sample T2...
Processing sample T3...
Processing sample T4...
Processing sample T5...
Processing sample T6...
Processing sample T7...
Processing sample T8...
Processing sample T9...
Processing sample T10...
Processing sample T11...
Processing sample T12...
Processing sample T13...
Processing sample T14...
Processing sample T15...
Processing sample T16...
Processing sample T17...
Processing sample T18...
Processing sample T19...
Processing sample T20...
Processing sample T21...
Processing sample T22...
Processing sample T23...
Processing sample T24...
Processing sample T25...
Processing sample T26...
Processing sample T27...
Processing sample T28...
Processing sample T29...
Processing sample T30...
Processing sample T31...
Processing sample T32...
Processing sample T33...
Processing sample T34...
Processing sample T35...
Processing sample T36...
Processing sample T37...
Processing sample T38...
Processing sample T39...
Processing sample T40...
Processin

In [4]:
# Define the directories
sample_dir = "E:\A files\Master\Dissertation\Samples\WYT\WYT_ply"
output_dir = "E:\A files\Master\Dissertation\Samples\WYT"

# Initialize and run the comparer
total_samples = 344
comparer = PointCloudComparer(sample_dir, output_dir)
comparer.process_all(total_samples)

Processing sample T148...
Processing sample T149...
Processing sample T150...
Processing sample T151...
Processing sample T152...
Processing sample T153...
Processing sample T154...
Processing sample T155...
Processing sample T156...
Processing sample T157...
Processing sample T158...
Processing sample T159...
Processing sample T160...
Processing sample T161...
Processing sample T162...
Processing sample T163...
Processing sample T164...
Processing sample T165...
Processing sample T166...
Processing sample T167...
Processing sample T168...
Processing sample T169...
Processing sample T170...
Processing sample T171...
Processing sample T172...
Processing sample T173...
Processing sample T174...
Processing sample T175...
Processing sample T176...
Processing sample T177...
Processing sample T178...
Processing sample T179...
Processing sample T180...
Processing sample T181...
Processing sample T182...
Processing sample T183...
Processing sample T184...
Processing sample T185...
Processing s

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\A files\\Master\\Dissertation\\Samples\\WYT\\WYT_ply\\T492_M.ply'